Use _ to indicate these variables are private to the class.  
To remind myself not to change them directly in code outside the class.  
Instead, use the class's methods, which check for consequences (validation etc) of a variable change.  

In [18]:
from random import randint

# Dictionary to map Card face to Numerical value
cards_dict = {'2':2,
          '3':3,
          '4':4,
          '5':5,
          '6':6,
          '7':7,
          '8':8,
          '9':9,
          '10':10,
          'J':10,
          'Q':10,
          'K':10,
          'A':11} # We'll handle soft Aces in code.

class Player:
    
    # initialize the attributes
    def __init__(self, name):
        self.name = name 
#         self.hand = [] # This gets reset at the top of each game
        self.status = 'open' # 'open', 'win', 'stand' (includes push), lose' (bust, dealer natural blackjack)	
    
    def print_hand(self):
        hand_formatted = ' '.join(self.hand)
        print('{}: {}'.format(self.name, hand_formatted))
                
    
    def calculate_soft_score(self):
        ''' Try to convert some/all Aces to 1 to get a better score. 
        Return closest to 21 without going over.'''
        
        soft_score = self.hard_score
        n_aces = len([card for card in self.hand if card == 'A'])
        
        if n_aces>0:
            for ace in range(1, n_aces+1):
                soft_score -= 10 #  -11 + 1
                if soft_score <=21: # Stop as soon as we get closest without going over.
                    break
                    
        return soft_score
    
    def evaluate_hand(self):
        ''' Update soft and hard scores, and detect blackjacks and busts.'''

        # Evaluate hard score (all Aces = 11)
        self.hard_score = 0 
        
        for card in self.hand:
            self.hard_score += cards_dict[card]
        
        # Hard blackjack?
        if self.hard_score == 21: 
            self.best_score = 21 

        # Else, if hard_score is not a bust, it's our best score (closet to 21 without going over).
        elif self.hard_score<21:
            self.best_score = self.hard_score
        
        # Else, hard_score is a bust, so calculate the soft score (try some/all Aces = 1)
        else: 
            self.best_score = self.calculate_soft_score()                        
            
            if self.best_score == 21:
                self.status = 'blackjack'            
                
            if self.best_score>21:  # Soft score didn't help
                self.status = 'bust'


class Human(Player):

    def move(self, tbl): # inelegant to pass table here. and to have to name it tbl. Better is to put move in Table, with subroutnine here
        decision = ''
        while (decision != 'h') and (decision != 's'):
            decision = input("Hit (h) or Stand (s)?")
        if decision == 'h':
            self.hand += tbl.draw()
            self.print_hand()
            self.evaluate_hand()
        if decision == 's':
            self.status = 'stand'

class Dealer(Player):
    
    def move(self, tbl):
        self.status = 'bust' # PLACEHOLDER. ADD RULES
    
    def print_hand(self, is_deal=False): 
        '''Override the regular Player print function to handle the hidden Dealer card at deal.'''
        
        if is_deal: 
            hand_formatted = str(self.hand[0]) + ' ' + '?'
        else:
            hand_formatted = ' '.join(self.hand)
        
        print('{}: {}'.format(self.name, hand_formatted))

class Table():
            
    def _new_deck(self):
        self._deck = list(cards_dict)*4

    def __init__(self):
        
        self.is_active = True
        
        # Create a new deck
        self._new_deck()

        # Add humans
        player_names = []
        while not player_names: # While is to handle empty input
            player_names = input("What's your name?").split(',') 
            
        self.players = [Human(name) for name in player_names]

        # Add dealer
        self.players += [Dealer('Encore')]
                
    def draw(self):
        ''' Takes 1 card out of the deck. Depletes deck by 1.'''
        
        l_deck =len(self._deck)
        
        # If we're out of cards, get a new deck.
        if l_deck <0: 
            self._new_deck()

        new_card = self._deck[randint(0, l_deck-1)] # Shuffles when drawing a card. Inelegant. Fix later.
        self._deck.remove(new_card)
        return([new_card])

    def deal(self, player_to_deal_to):
        ''' Draw two cards.'''
        player_to_deal_to.hand += self.draw() + self.draw()

    def end_game(self):
        # Show dealer's full hand.
        self.players[-1].print_hand()
        dealer_status = self.players[-1].status
        dealers_score = self.players[-1].best_score
        
        # Evaluate how each non-dealer player did. 
        for p in self.players[0:-1]: # 
            # If player busted/blackjacked already, and Dealer didn't have a natural blackjack, player's status will already be final. 
            
            # Dealer has blackjack. (Includes natural blackjack)
            if dealer_status == 'blackjack': 
                if p.status == 'blackjack': # So does player. 
                    p.status = 'push'
                else: # Player loses (including on a natural blackjack)
                    p.status = 'lost'
                      
            # Dealer busted, player didn't. I win!
            elif (dealer_status == 'bust') and (p.status!='bust'):
                p.status = 'won'
            
            # Both player and Dealer are <=21
            elif p.best_score >= dealers_score: # CONTINUE
                if p.best_score >= dealers_score:
                    p.status = 'won'
                else:
                    p.status = 'pushed'
                
            print("{} has {}!".format(p.name, p.status))
        
    def run_game(self):

        for p in self.players:
            p.status = 'open' 
            p.hand = []
            
            self.deal(p)
            
            if isinstance(p, Dealer): # This is inelegant. Just always pass is_deal?
                p.print_hand(is_deal = True)
            else:
                p.print_hand()
            p.evaluate_hand()

        # Does dealer have a natural blackjack?
        if self.players[-1].status == 'blackjack':
            self.end_game()

        # Otherwise, play out the hands of each player, one at a time.
        else:
            for p in self.players: 
                while p.status == 'open':
                    p.move(self) # feels inelegant
        
        self.end_game()
        

In [19]:
table = Table()

while table.is_active:
    table.run_game()
    table.is_active = input('Play again? (y)') == 'y'

What's your name? Chad


Chad: 8 J
Encore: A ?


Hit (h) or Stand (s)? s


Encore: A J
Chad has won!


Play again? (y) y


Chad: A J
Encore: 3 ?


Hit (h) or Stand (s)? s


Encore: 3 A
Chad has won!


Play again? (y) n


In [ ]:
%debug